Below are the scripts we ran using the Differential Language Analysis ToolKit (DLATK) to conduct our analysis for the paper: Analyzing Cardiovascular Health Outcomes Across American Communities Using Twitter/X: A Data Validation Study.

Reference Link: https://dlatk.github.io/dlatk/

In [ ]:
#Obtaining Topics Significantly associated with Behaviors
#Create ngrams for messages:
./dlatkInterface.py -d covid_cv_reddit -t cleaned_acp -c message_id --add_ngrams -n 1 --feat_occ_filter
#Create LDA topics:
./dlatkInterface.py -d covid_cv_reddit -t cleaned_acp -c message_id -f 'feat$1gram$cleaned_acp$message_id$16to16$0_01' --estimate_lda_topics --lda_lexicon_name cleaned_topics --mallet_path /home/sharath/mallet-2.0.8/bin/mallet --num_topics 100 --num_stopwords 50 --num_lda_threads 100 --save_lda_file /home/sukanyaj/cleaned_topics
#Extract topics at message_level:
./dlatkInterface.py -d covid_cv_reddit -t cleaned_acp -c message_id --add_lex_table -l cleaned_topics_cp --weighted_lexicon
#Run Topic DLA for behaviors
./dlatkInterface.py -d covid_cv_reddit -t cleaned_acp -c message_id -f 'feat$cat_cleaned_topics_cp_w$cleaned_acp$message_id$1gra' --outcome_table cleaned_acp --outcomes is_pa is_diet is_smoking is_sleep is_substance_use is_alcohol --output cleaned_topics --topic_tagcloud --make_topic_wordcloud --topic_lexicon cleaned_topics_freq_t50ll --group_freq_thresh 1 --tagcloud_colorscheme bluered --csv --logistic --keep_low_variance_outcomes
#Top messages per topic
./dlatkInterface.py -d covid_cv_reddit -t cleaned_acp -c message_id  -f 'feat$cat_cleaned_topics_cp_w$cleaned_acp$message_id$1gra' --output top_messages_100_topics --top_messages 20 --lex_table cleaned_topics_freq_t50ll --group_freq_thresh 6

#10-fold Ridge Regression Cross-Validation to assess the Predictive Utility of LDA topics
./dlatkInterface.py -d covid_cv_reddit -t combined_acp_dedup -c FIPS \
-f 'feat$cat_cleaned_topics_cp_w$combined_acp_dedup$FIPS$1gra' \
--outcome_table health_factors  --group_freq_thresh 500 \
--outcomes leisure smoking sleeping obesity cholesterol_screening high_chol diabetes medicine blood_pressure --outcome_controls LogIncome EducationIndex --output_name health_measure_output_pred  \
--combo_test_reg --model ridgecv --folds 10